In [1]:
import wandb
from tqdm import trange
import os

In [2]:
project_name = "Memorization_Intermediate_Checkpoints_From_Pretraining"

In [3]:
!wandb login

wandb: Currently logged in as: aflah. Use `wandb login --relogin` to force relogin


In [4]:
project = wandb.Api().project(project_name)

In [5]:
api = wandb.Api()
entity, project = "aflah", project_name
runs = api.runs(entity + "/" + project)

In [6]:
save_path = "..\data\intmed_ckpt_loss_data"

In [7]:
os.listdir(save_path)

['non_random_strings pythia-1b_0 full-finetuning 1024x1 Epochs- 100 BSize- 1 Seed- 42',
 'non_random_strings pythia-1b_0 full-finetuning 128x1 Epochs- 100 BSize- 1 Seed- 42',
 'non_random_strings pythia-1b_0 full-finetuning 16x1 Epochs- 100 BSize- 1 Seed- 42',
 'non_random_strings pythia-1b_0 full-finetuning 256x1 Epochs- 100 BSize- 1 Seed- 42',
 'non_random_strings pythia-1b_0 full-finetuning 32x1 Epochs- 100 BSize- 1 Seed- 42',
 'non_random_strings pythia-1b_0 full-finetuning 512x1 Epochs- 100 BSize- 1 Seed- 42',
 'non_random_strings pythia-1b_0 full-finetuning 64x1 Epochs- 100 BSize- 1 Seed- 42',
 'non_random_strings pythia-1b_1 full-finetuning 1024x1 Epochs- 100 BSize- 1 Seed- 42',
 'non_random_strings pythia-1b_1 full-finetuning 128x1 Epochs- 100 BSize- 1 Seed- 42',
 'non_random_strings pythia-1b_1 full-finetuning 16x1 Epochs- 100 BSize- 1 Seed- 42',
 'non_random_strings pythia-1b_1 full-finetuning 256x1 Epochs- 100 BSize- 1 Seed- 42',
 'non_random_strings pythia-1b_1 full-finetun

In [8]:
loss_res = {}
for r in trange(len(runs)):
    run = runs[r]
    if run.state != "finished":
        continue

    save_at = os.path.join(save_path, run.name)
    # replace : with - to avoid errors
    save_at = save_at.replace(":", "-")
    if os.path.exists(save_at):
        # print(f"Skipping {run.name} as it already exists")
        continue
    history = run.history()
    loss_val = history['eval/loss'].tolist()
    loss_res[run.name] = loss_val
    with open(save_at, "w") as f:
        f.write(str(loss_val))


100%|██████████| 2156/2156 [01:41<00:00, 21.24it/s] 


In [9]:
# read all the files and get the loss values
import os

def read_file(file):
    with open(file, "r") as f:
        data = f.read()
    return data

def parse_loss(loss):
    loss = loss.replace("[", "")
    loss = loss.replace("]", "")
    loss = loss.replace(",", "")
    loss = loss.split(" ")
    loss = [float(l) for l in loss]
    return loss

save_path = "..\data\intmed_ckpt_loss_data"
files = os.listdir(save_path)
losses = {}

for file in files:
    file_path = os.path.join(save_path, file)
    loss = read_file(file_path)
    loss = parse_loss(loss)
    losses[file] = loss


In [26]:
# remove nan numric values
import numpy as np

for k, v in losses.items():
    losses[k] = [l for l in v if not np.isnan(l)]

In [27]:
# convert to dataframe
import pandas as pd

df = pd.DataFrame(losses)

df.to_csv("../data/losses_memorization_intermediate_checkpoints.csv", index=False)

In [28]:
df

,non_random_strings pythia-1b_0 full-finetuning 1024x1 Epochs- 100 BSize- 1 Seed- 42,non_random_strings pythia-1b_0 full-finetuning 128x1 Epochs- 100 BSize- 1 Seed- 42,non_random_strings pythia-1b_0 full-finetuning 16x1 Epochs- 100 BSize- 1 Seed- 42,non_random_strings pythia-1b_0 full-finetuning 256x1 Epochs- 100 BSize- 1 Seed- 42,non_random_strings pythia-1b_0 full-finetuning 32x1 Epochs- 100 BSize- 1 Seed- 42,non_random_strings pythia-1b_0 full-finetuning 512x1 Epochs- 100 BSize- 1 Seed- 42,non_random_strings pythia-1b_0 full-finetuning 64x1 Epochs- 100 BSize- 1 Seed- 42,non_random_strings pythia-1b_1 full-finetuning 1024x1 Epochs- 100 BSize- 1 Seed- 42,non_random_strings pythia-1b_1 full-finetuning 128x1 Epochs- 100 BSize- 1 Seed- 42,non_random_strings pythia-1b_1 full-finetuning 16x1 Epochs- 100 BSize- 1 Seed- 42,...,random_strings pythia-1b_98000 full-finetuning 32x1 Epochs- 100 BSize- 1 Seed- 42,random_strings pythia-1b_98000 full-finetuning 512x1 Epochs- 100 BSize- 1 Seed- 42,random_strings pythia-1b_98000 full-finetuning 64x1 Epochs- 100 BSize- 1 Seed- 42,random_strings pythia-1b_99000 full-finetuning 1024x1 Epochs- 100 BSize- 1 Seed- 42,random_strings pythia-1b_99000 full-finetuning 128x1 Epochs- 100 BSize- 1 Seed- 42,random_strings pythia-1b_99000 full-finetuning 16x1 Epochs- 100 BSize- 1 Seed- 42,random_strings pythia-1b_99000 full-finetuning 256x1 Epochs- 100 BSize- 1 Seed- 42,random_strings pythia-1b_99000 full-finetuning 32x1 Epochs- 100 BSize- 1 Seed- 42,random_strings pythia-1b_99000 full-finetuning 512x1 Epochs- 100 BSize- 1 Seed- 42,random_strings pythia-1b_99000 full-finetuning 64x1 Epochs- 100 BSize- 1 Seed- 42
0,11.016693,10.992131,11.091809,11.011527,11.035279,11.017520,10.945030,11.016693,10.992131,11.091809,...,9.522458,7.863572,8.738505,7.599267,8.716075,10.126730,8.307323,9.178914,8.037970,8.568211
1,8.901055,7.224306,2.611040,7.924487,4.526464,8.535429,6.111122,8.901055,7.224306,2.611040,...,3.763808,3.514955,3.808859,3.546556,4.704615,4.237081,4.170257,4.792204,3.752864,4.886656
2,8.210053,4.345475,0.146715,7.378366,0.608916,7.501762,2.337476,8.210053,4.345475,0.146715,...,2.891381,3.282128,2.995453,3.319060,3.329486,2.775464,3.306349,3.205171,3.327850,3.471475
3,7.760817,3.405970,0.199011,5.210966,0.389684,7.103293,0.491823,7.760817,3.405970,0.199011,...,2.698118,3.158076,3.046290,3.297833,3.055157,2.658914,3.146116,3.256309,3.248203,3.039637
4,7.220320,1.316326,0.751253,4.698982,0.564386,6.227464,0.215208,7.220320,1.316326,0.751253,...,2.276920,3.077721,2.741960,3.212463,2.818082,1.985834,2.995611,2.620466,3.203408,2.718083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,0.434043,0.000233,0.000014,0.005887,0.000025,0.095608,0.000051,0.434043,0.000233,0.000014,...,0.000002,0.000014,0.000003,0.000042,0.000009,0.000002,0.000017,0.000006,0.000029,0.000002
97,0.431303,0.000232,0.000014,0.005714,0.000025,0.094017,0.000051,0.431303,0.000232,0.000014,...,0.000002,0.000014,0.000003,0.000042,0.000009,0.000002,0.000017,0.000006,0.000029,0.000002
98,0.429193,0.000232,0.000014,0.005436,0.000025,0.092365,0.000051,0.429193,0.000232,0.000014,...,0.000002,0.000014,0.000003,0.000042,0.000009,0.000002,0.000017,0.000006,0.000029,0.000002
99,0.427564,0.000231,0.000014,0.005227,0.000025,0.091355,0.000051,0.427564,0.000231,0.000014,...,0.000002,0.000014,0.000003,0.000042,0.000009,0.000002,0.000017,0.000006,0.000029,0.000002


In [30]:
import plotly.express as px

def get_col_name(str_type, size, ckpt):
    return f"{str_type} pythia-1b_{ckpt} full-finetuning {size}x1 Epochs- 100 BSize- 1 Seed- 42"

str_types = ['non_random_strings', 'random_strings']
sizes = [16, 32, 64, 128, 256, 512, 1024]
ckpts = [0,1,4,16,64,256,1000, 2000, 10000, 50000, 100000, 143000]
# for i in range(2000, 144000, 1000):
#     ckpts.append(i)

for str_type in str_types:
    for size in sizes:
        col_names = [get_col_name(str_type, size, ckpt) for ckpt in ckpts]
        subset_df = df[col_names]
        # plot
        fig = px.line(subset_df, y=col_names, x=subset_df.index)
        # set x and y axis labels
        fig.update_layout(
            xaxis_title="Epoch",
            yaxis_title="Loss",
        )
        # change name in legend
        fig.for_each_trace(lambda t: t.update(name=t.name.split(" ")[1]))

        # title
        fig.update_layout(title=f"Loss for {str_type} of size {size}")

        # change legend title to Layer
        fig.update_layout(legend_title_text='Layer')
        fig.show()